# Function as First Citizen

_Python_ considers function as object, not as a built-in and language core feature, but as an object programer can handle. A kind of variable with a behavior.

You can make the same vith a block of behavior: a function...

As you can says :

_a_ is an integer
```python
a=1
```
_b_ is a string
```python
b='hi!'
```
or _c_ is Plane()
```python
c=Plane()
```

You could says _cube_ is a block of behavior
```python
def cube(n)
    return n*n*n
```

Let's try do this: 

In [ ]:
def cube(n):
    return n*n*n

In [ ]:
print(cube(2), cube(5))

This function has properties

In [ ]:
'/'.join(dir(cube))

This function can be assigned to another variable as _foo_.

In [ ]:
foo = cube

print(foo(3), foo.__name__)

Passed as an argument:

In [ ]:
def apply_to_numbers(fn,*numbers):
    for n in numbers:
        print(f'{fn.__name__}({n}) ->  {fn(n)}')

apply_to_numbers(cube,1,2,5,250)


It could also _deleted_ or _unset_

In [ ]:
def foo():
    print("foo")
foo()

In [ ]:
del foo
try:B
    foo()
    raise Exception('Must fail')
except NameError:
    pass

A function could be returned as a regular value

In [ ]:
import types

def create_upper_function():
    def bar(x):
        return x.upper()
    return bar

fn  = create_upper_function()
assert fn('dog') == 'DOG'
assert fn('cat') == 'CAT'
assert isinstance(fn, types.FunctionType)

Although we created the `bar` function, it is undefined _outside_ of `create_upper_function`.

Its scope is, as a regular variable, limited to the body of the outer function.

In [ ]:
try:
    assert bar is not None
    raise Exception('It must not have been raised')
except NameError:
    pass

## Advantages of this 

With this feature you can **manage** behaviors as simple variables and apply some well-known patterns.

## Space craft command module

Imagine we have to define a control module for a aquatic drone.

We sent tele commands (TC) to this drone and it moves according to them.

There's three available

* Heading, whose quantity in an angle (degres)
* Distance to advance into the current heading (meters)
* Depth to go up and down (meters) 

Commands are sent by as bunch as :

_h:122.97|a:200|h:125|a:20_

In traditional language we would have this (pseudo language)

```
sequence = CONVERT_TO_COMMAND(tc)

FOR EACH (command,quantity) IN sequence
BEGIN
    IF command == 'heading' SET_HEADING(quantity)
    IF command == 'advance' ADVANCE(quantity)
    IF command == 'depth' SET_DEPTH(quantity)
END
```

In *python*, we could do differently.

In [ ]:
tc = 'h:122.97|a:200|h:125|a:20'
print(tc.split('|'))
[tuple(sequence.split(':')) for sequence in tc.split('|')]


In [ ]:
# Define commands

def set_heading_to(angle):
    print('HEADING TO ',angle)

def advance(distance):
    print('ADVANCE TO ', distance)

def set_depth(distance):
    print('DEPTH TO', distance)

# Map them to a TC letter
mapping = {
    'h': set_heading_to,
    'a': advance,
    'd': set_depth
}

In [ ]:
# Run TC
def run_tc(mappinging, tc):
    
    commands = [tuple(sequence.split(':')) for sequence in tc.split('|')]
    
    for command in commands:
        type_command = command[0]
        quantity_command = command[1]
        fn = mapping[type_command]
        
        fn(quantity_command)
        
run_tc(mapping, 'h:122.97|a:200|h:125|a:20')

This method allow to decouple the `run_tc` from the available commands.

If we want to add a new command as *incline(angle)*, no problem

In [ ]:
def incline(angle):
    print('ANGLE TO ',angle)
    
mapping['i'] = incline

run_tc(mapping, 'h:122.97|i:12|h:125|a:20')

## Chain functions

Imagine we have a lot of conditions to check about a password for exemple.

It must :

- has a lenght between 3 and 10 
- contains a upper case character at least


In [ ]:
def check_has_at_least_an_uppercase(s):
    return any(c.isupper() for c in s)

def check_len(s):
    return 3<= len(s) <= 10

# - usual version

def checking(s):
    return check_has_at_least_an_uppercase(s) and check_len(s)

assert checking('FOO')
assert not checking('Fo')
assert checking('Foo')
assert not checking('foo')



Bt what if we want to add another checking has

* contains a digit at least



In [ ]:
def check_has_a_least_one_digit(s):
    return any(c.isdigit() for c in s)

We have to change `checking`...

In [ ]:
def checking(s):
    return check_has_at_least_an_uppercase(s) and check_len(s) and check_has_a_least_one_digit(s)

The best possibility is to apply a pipeline of any ` check_*` functions.

In [ ]:
import sys
current_module = sys.modules[__name__]
names= [name for name in dir(current_module) if name.startswith('check_')]

def checking(s):
    return all(getattr(current_module, name).__call__(s) for name in names)

In [ ]:
assert not checking('foo')

In [ ]:
assert checking('Foo1')